In [ ]:
# 구글드라이브연동
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 작업디렉토리 설정 (voicephishing1폴더)
import os

# 작업할 디렉토리 설정
work_dir = '/content/drive/My Drive/voicephishingDetection'

# 작업 디렉토리로 이동
os.chdir(work_dir)

# 현재 작업 디렉토리 확인
print("현재 작업 디렉토리:", os.getcwd())


현재 작업 디렉토리: /content/drive/My Drive/voicephishingDetection


In [ ]:
# 7차 lstm학습코드 -> 시퀀스 30으로 제한 train_data.csv , test_data.csv, val_data.csv파일로 사전에 나눔. (label밸런싱, call_id기준으로 적용)
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# -------------------------
# 1. 학습 데이터 로딩 및 전처리
# -------------------------
train_df = pd.read_csv('train_data.csv', encoding='utf-8')

train_texts = train_df['transcript'].astype(str).values
train_labels = train_df['label'].values

# -------------------------
# 2. 검증 데이터 로딩 및 전처리
# -------------------------
valid_df = pd.read_csv('val_data.csv', encoding='utf-8')

valid_texts = valid_df['transcript'].astype(str).values
valid_labels = valid_df['label'].values

# -------------------------
# 3. 토크나이저 및 시퀀스 변환 (train 기준으로 학습)
# -------------------------
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train = pad_sequences(tokenizer.texts_to_sequences(train_texts), maxlen=30, padding='post', truncating='post')
y_train = np.array(train_labels)

X_valid = pad_sequences(tokenizer.texts_to_sequences(valid_texts), maxlen=30, padding='post', truncating='post')
y_valid = np.array(valid_labels)

# -------------------------
# 4. LSTM 모델 구성
# -------------------------
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=30))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# -------------------------
# 5. 모델 학습 (validation 포함)
# -------------------------
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_valid, y_valid))

# -------------------------
# 6. tokenizer, 모델 저장
# -------------------------
with open('/content/drive/MyDrive/voicephishingDetection/tokenizer0514.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

model.save('/content/drive/MyDrive/voicephishingDetection/lstm_model0514.keras')

print("✅ 모델 학습 및 검증 완료, 저장도 완료")

# -------------------------
# 7. 검증 데이터 성능 출력
# -------------------------
val_loss, val_acc = model.evaluate(X_valid, y_valid)
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
348/348 ━━━━━━━━━━━━━━━━━━━━ 47s 119ms/step - accuracy: 0.8942 - loss: 0.2282 - val_accuracy: 0.9921 - val_loss: 0.0261
Epoch 2/5
348/348 ━━━━━━━━━━━━━━━━━━━━ 81s 115ms/step - accuracy: 0.9941 - loss: 0.0200 - val_accuracy: 0.9903 - val_loss: 0.0246
Epoch 3/5
348/348 ━━━━━━━━━━━━━━━━━━━━ 41s 117ms/step - accuracy: 0.9964 - loss: 0.0110 - val_accuracy: 0.9803 - val_loss: 0.0358
Epoch 4/5
348/348 ━━━━━━━━━━━━━━━━━━━━ 39s 111ms/step - accuracy: 0.9963 - loss: 0.0114 - val_accuracy: 0.9918 - val_loss: 0.0221
Epoch 5/5
348/348 ━━━━━━━━━━━━━━━━━━━━ 40s 108ms/step - accuracy: 0.9984 - loss: 0.0057 - val_accuracy: 0.9907 - val_loss: 0.0227
✅ 모델 학습 및 검증 완료, 저장도 완료
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9889 - loss: 0.0188
Validation Accuracy: 0.9907
Validation Loss: 0.0227


In [ ]:
# 검증 라벨 분포 확인
print(valid_df['label'].value_counts())

# call_id 겹치는지 확인
train_ids = set(train_df['call_id'].unique())
valid_ids = set(valid_df['call_id'].unique())
print("겹치는 call_id 수:", len(train_ids & valid_ids))

In [ ]:
# 7차 lstm학습모델 검증 -> test_data.csv
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# -------------------------------
# 1. 테스트 데이터 로딩
# -------------------------------
test_df = pd.read_csv('/content/drive/MyDrive/voicephishing1/test_data.csv', encoding='utf-8-sig')
test_texts = test_df['transcript'].astype(str).values
test_labels = test_df['label'].values
call_ids = test_df['call_id'].values  # call_id 추가

# -------------------------------
# 2. 저장된 Tokenizer 로드
# -------------------------------
with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# -------------------------------
# 3. 텍스트 → 시퀀스 변환 및 패딩
# -------------------------------
X_test = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(X_test, maxlen=30, truncating='post', padding='post')
y_test = np.array(test_labels)

# -------------------------------
# 4. 저장된 모델 로드
# -------------------------------
model = load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')

# -------------------------------
# 5. 개별 문장의 예측값 (확률) 구하기
# -------------------------------
predictions = model.predict(X_test)  # 예측값 (0과 1 사이의 확률)

# -------------------------------
# 6. call_id별로 예측 확률의 평균값 계산
# -------------------------------
test_df['prediction'] = predictions  # 예측 확률을 DataFrame에 추가

# call_id별로 평균 예측 확률 계산 (각 통화마다 평균 확률)
call_id_avg_predictions = test_df.groupby('call_id')['prediction'].mean()

# -------------------------------
# 7. 각 통화에 대해 보이스피싱 여부 판단
# -------------------------------
# 예: 평균 확률이 0.5 이상이면 피싱 통화로 판단
call_id_avg_predictions = call_id_avg_predictions.apply(lambda x: 1 if x >= 0.5 else 0)

# -------------------------------
# 8. 전체 성능 평가 (전체 통화 기준으로 평가)
# -------------------------------
# 테스트셋의 실제 라벨과 예측 라벨을 비교
true_labels = test_df.groupby('call_id')['label'].first()  # 통화별 첫 번째 라벨로 참값 선택
pred_labels = call_id_avg_predictions  # 통화별 예측값

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 성능 평가 지표 출력
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)

print(f"✅ 정확도: {accuracy:.4f}")
print(f"✅ 정밀도: {precision:.4f}")
print(f"✅ 재현율: {recall:.4f}")
print(f"✅ F1-Score: {f1:.4f}")


In [ ]:
# 7차 lstm학습모델 과적합 여부 판단 1
import random
import pickle
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 모델과 Tokenizer 로딩
def load_model_and_tokenizer():
    # LSTM 모델과 Tokenizer 로드
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

# 기존 데이터 복사
test_df_flipped = test_df.copy()

# 통화(call_id) 단위로 라벨 일부 뒤집기 (예: 10%)
call_ids = test_df_flipped['call_id'].unique()
num_to_flip = max(1, int(len(call_ids) * 0.1))
flipped_call_ids = random.sample(list(call_ids), num_to_flip)

# call_id 단위로 라벨 반전
for cid in flipped_call_ids:
    original_label = test_df_flipped.loc[test_df_flipped['call_id'] == cid, 'label'].iloc[0]
    test_df_flipped.loc[test_df_flipped['call_id'] == cid, 'label'] = 1 - original_label

print(f"🔄 라벨 뒤집은 통화 수: {num_to_flip}")

# -------------------------------
# 1. 모델과 Tokenizer 로드
# -------------------------------
model, tokenizer = load_model_and_tokenizer()

# -------------------------------
# 2. 라벨 반전된 데이터에 대해 예측 (각 통화별로)
# -------------------------------
# test_df_flipped에서 'call_id'별로 음성 텍스트를 처리하여 예측
flipped_texts = test_df_flipped['transcript'].astype(str).values
X_flipped = tokenizer.texts_to_sequences(flipped_texts)
X_flipped = pad_sequences(X_flipped, maxlen=30, truncating='post', padding='post')

# 예측값 (0과 1 사이의 확률)
flipped_predictions = model.predict(X_flipped)

# -------------------------------
# 3. call_id별로 예측 확률의 평균값 계산
# -------------------------------
test_df_flipped['prediction'] = flipped_predictions  # 예측 확률을 DataFrame에 추가
call_id_avg_predictions_flipped = test_df_flipped.groupby('call_id')['prediction'].mean()

# -------------------------------
# 4. call_id별로 라벨 반전 후 예측값 평가
# -------------------------------
# 다시 그룹핑 및 평가
true_labels_flipped = test_df_flipped.groupby('call_id')['label'].first()  # 통화별 첫 번째 라벨로 참값 선택
pred_labels_flipped = call_id_avg_predictions_flipped.apply(lambda x: 1 if x >= 0.5 else 0)  # 0.5 이상이면 피싱 통화로 판단

# 성능 평가 지표 출력
accuracy = accuracy_score(true_labels_flipped, pred_labels_flipped)
precision = precision_score(true_labels_flipped, pred_labels_flipped)
recall = recall_score(true_labels_flipped, pred_labels_flipped)
f1 = f1_score(true_labels_flipped, pred_labels_flipped)

print("\n✅ [라벨 반전 테스트]")
print(f"정확도: {accuracy:.4f}")
print(f"정밀도: {precision:.4f}")
print(f"재현율: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


In [ ]:
# 7차 lstm학습모델 과적합 여부 판단 2
import random
import tensorflow as tf
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 모델과 Tokenizer 로딩 함수
def load_model_and_tokenizer():
    # LSTM 모델과 Tokenizer 로드
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

# 텍스트 섞기 함수
def shuffle_words(text):
    words = text.split()
    if len(words) > 1:
        random.shuffle(words)
    return ' '.join(words)

# 모델과 Tokenizer 로드
model, tokenizer = load_model_and_tokenizer()

# 테스트 데이터 복사 및 텍스트 일부 랜덤 변경
test_df_perturbed = test_df.copy()
perturb_rate = 0.1  # 10%만 섞기

# 텍스트 섞기
indices_to_perturb = random.sample(range(len(test_df_perturbed)), int(len(test_df_perturbed) * perturb_rate))
test_df_perturbed.loc[indices_to_perturb, 'transcript'] = test_df_perturbed.loc[indices_to_perturb, 'transcript'].apply(shuffle_words)

# 텍스트 시퀀스 변환 및 패딩
X_perturbed = tokenizer.texts_to_sequences(test_df_perturbed['transcript'].astype(str).values)
X_perturbed = pad_sequences(X_perturbed, maxlen=30, truncating='post', padding='post')

# 예측
perturbed_predictions = model.predict(X_perturbed)
test_df_perturbed['prediction'] = perturbed_predictions

# 통화별 평균 예측값 계산
perturbed_call_avg = test_df_perturbed.groupby('call_id')['prediction'].mean()
perturbed_call_avg = perturbed_call_avg.apply(lambda x: 1 if x >= 0.5 else 0)

# 평가
true_labels = test_df_perturbed.groupby('call_id')['label'].first()
pred_labels = perturbed_call_avg

accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)

print("\n✅ [텍스트 섞기 테스트]")
print(f"정확도: {accuracy:.4f}")
print(f"정밀도: {precision:.4f}")
print(f"재현율: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

In [ ]:
!pip install openai-whisper

In [ ]:
# # 7차 lstm학습모델 그놈목소리 수사기관 사칭형 mp3파일 테스트 (음성 시간 분할 없이 테스트)
import os
from pydub import AudioSegment
import whisper
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# -------------------------------
# 1. mp3 -> wav 변환
# -------------------------------
def mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")
    print(f"MP3 파일이 {wav_file_path}로 변환되었습니다.")
    return wav_file_path

# -------------------------------
# 2. Whisper로 음성 인식 (STT)
# -------------------------------
def transcribe_audio_with_whisper(wav_file_path):
    model = whisper.load_model("large")  # Whisper의 모델 로드
    result = model.transcribe(wav_file_path)
    print(f"음성 인식 결과: {result['text']}")
    return result['text']

# -------------------------------
# 3. 텍스트 처리 (Tokenizer 로딩, 시퀀스 변환 및 패딩)
# -------------------------------
def preprocess_text(text, tokenizer, max_length=30):
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequences, maxlen=max_length, truncating='post', padding='post')
    return padded_sequence

# -------------------------------
# 4. LSTM 모델로 예측
# -------------------------------
def predict_with_lstm(model, text, tokenizer):
    processed_text = preprocess_text(text, tokenizer)
    prediction = model.predict(processed_text)
    return prediction

# -------------------------------
# 5. 모델 및 Tokenizer 로딩
# -------------------------------
def load_model_and_tokenizer():
    # LSTM 모델과 Tokenizer 로드
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

# -------------------------------
# 6. MP3 파일 경로 지정 후 처리
# -------------------------------
def process_mp3(mp3_file_path):
    # 1. mp3 파일을 wav로 변환
    wav_file_path = mp3_to_wav(mp3_file_path, 'temp_audio.wav')

    # 2. Whisper로 음성 인식 (STT)
    transcribed_text = transcribe_audio_with_whisper(wav_file_path)

    # 3. 모델과 Tokenizer 로드
    model, tokenizer = load_model_and_tokenizer()

    # 4. 예측
    prediction = predict_with_lstm(model, transcribed_text, tokenizer)
    print(f"예측 결과 (피싱 통화 확률): {prediction[0][0]:.4f}")

    # 결과 반환
    return transcribed_text, prediction

# -------------------------------
# 7. 예시: mp3 파일 처리
# -------------------------------
mp3_file_path = '/content/drive/MyDrive/voicephishing1/example.mp3'  # 처리할 mp3 파일 경로
transcribed_text, prediction = process_mp3(mp3_file_path)
print(f"Transcribed Text: {transcribed_text}")
print(f"Prediction: {prediction[0][0]:.4f}")  # 예측 확률 출력


In [ ]:
# # 7차 lstm학습모델 그놈목소리 수사기관 사칭형 mp3파일 테스트 (음성 시간 분할 30초 설정 테스트)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydub import AudioSegment
import whisper
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# -------------------------------
# 1. mp3 -> wav 변환
# -------------------------------
def mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")
    print(f"MP3 파일이 {wav_file_path}로 변환되었습니다.")
    return wav_file_path

# -------------------------------
# 2. Whisper로 음성 인식 (STT)
# -------------------------------
def transcribe_audio_with_whisper(wav_file_path, start_ms, duration_ms):
    model = whisper.load_model("large")  # Whisper의 모델 로드
    audio = AudioSegment.from_wav(wav_file_path)
    segment = audio[start_ms:start_ms + duration_ms]  # 구간별로 자르기
    segment.export("temp_segment.wav", format="wav")

    result = model.transcribe("temp_segment.wav")
    os.remove("temp_segment.wav")  # 임시 파일 삭제
    print(f"음성 인식 결과 (구간 {start_ms / 1000}s - {(start_ms + duration_ms) / 1000}s): {result['text']}")
    return result['text']

# -------------------------------
# 3. 텍스트 처리 (Tokenizer 로딩, 시퀀스 변환 및 패딩)
# -------------------------------
def preprocess_text(text, tokenizer, max_length=30):
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequences, maxlen=max_length, truncating='post', padding='post')
    return padded_sequence

# -------------------------------
# 4. LSTM 모델로 예측
# -------------------------------
def predict_with_lstm(model, text, tokenizer):
    processed_text = preprocess_text(text, tokenizer)
    prediction = model.predict(processed_text)
    return prediction

# -------------------------------
# 5. 모델 및 Tokenizer 로딩
# -------------------------------
def load_model_and_tokenizer():
    # LSTM 모델과 Tokenizer 로드
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

# -------------------------------
# 6. MP3 파일 경로 지정 후 처리
# -------------------------------
def process_mp3(mp3_file_path, segment_duration_ms=20000):  # 20초 구간으로 변경
    # 1. mp3 파일을 wav로 변환
    wav_file_path = mp3_to_wav(mp3_file_path, 'temp_audio.wav')

    # 2. 모델과 Tokenizer 로드
    model, tokenizer = load_model_and_tokenizer()

    # 3. 음성 파일의 총 길이 (밀리초 단위)
    audio = AudioSegment.from_wav(wav_file_path)
    total_length_ms = len(audio)

    # 4. 예측 결과 저장할 리스트
    time_intervals = []
    predictions = []

    # 5. 각 구간에 대해 Whisper로 음성 인식 후 예측 수행
    for start_ms in range(0, total_length_ms, segment_duration_ms):
        transcribed_text = transcribe_audio_with_whisper(wav_file_path, start_ms, segment_duration_ms)
        prediction = predict_with_lstm(model, transcribed_text, tokenizer)
        time_intervals.append(start_ms / 1000)  # 시간 (초 단위)
        predictions.append(prediction[0][0])  # 예측 확률 저장

    # 결과 반환
    return time_intervals, predictions

# -------------------------------
# 7. 예시: mp3 파일 처리 및 예측 결과 그래프 출력
# -------------------------------
mp3_file_path = '/content/drive/MyDrive/voicephishing1/example.mp3'  # 처리할 mp3 파일 경로
time_intervals, predictions = process_mp3(mp3_file_path)

# 그래프 시각화
plt.figure(figsize=(10, 6))
plt.plot(time_intervals, predictions, marker='o', color='b', linestyle='-', label='피싱 통화 확률')
plt.title("시간에 따른 피싱 통화 확률")
plt.xlabel("시간 (초)")
plt.ylabel("피싱 통화 확률")
plt.grid(True)
plt.show()

In [ ]:
import os
from pydub import AudioSegment
import whisper
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import string

# -------------------------------
# 1. mp3 → wav 변환
# -------------------------------
def mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")
    print(f"✅ MP3 → WAV 변환 완료: {wav_file_path}")
    return wav_file_path

# -------------------------------
# 2. Whisper 음성 인식
# -------------------------------
def transcribe_audio_with_whisper(wav_file_path):
    model = whisper.load_model("large")
    result = model.transcribe(wav_file_path)
    print("\n📝 음성 인식 결과:")
    print(result['text'])
    return result['text']

# -------------------------------
# 3. 전처리 함수
# -------------------------------
def preprocess_text(text, tokenizer, max_length=30):
    text = text.translate(str.maketrans('', '', string.punctuation))  # 문장부호 제거
    sequences = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    return padded

# -------------------------------
# 4. 예측 함수
# -------------------------------
def predict_with_lstm(model, text, tokenizer):
    padded_text = preprocess_text(text, tokenizer)
    prediction = model.predict(padded_text, verbose=0)
    return prediction[0][0]

# -------------------------------
# 5. 모델 & 토크나이저 로딩
# -------------------------------
def load_model_and_tokenizer():
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

# -------------------------------
# 6. 누적 위험도 계산 및 출력
# -------------------------------
def run_prediction_with_risk(text, model, tokenizer, base_threshold=0.5, detection_threshold=3.0):
    print("\n📊 누적 위험도 분석 시작")
    lines = [line.strip() for line in text.split('.') if line.strip()]  # 문장 분할
    cumulative_risk = 0.0
    phishing_detected_at = None
    predictions = []

    for i, line in enumerate(lines):
        prob = predict_with_lstm(model, line, tokenizer)
        predictions.append(prob)

        if prob > base_threshold:
            cumulative_risk += (prob - base_threshold)
        else:
            cumulative_risk = max(0, cumulative_risk - (base_threshold - prob))

        print(f"{i+1:02d}. \"{line}\" → 확률: {prob:.4f}, 누적 위험도: {cumulative_risk:.4f}")

        if phishing_detected_at is None and cumulative_risk >= detection_threshold:
            phishing_detected_at = i * 10  # 시간 기준 10초 단위로 가정

    if phishing_detected_at is not None:
        print(f"\n⚠️ 보이스피싱 의심 탐지 시점: {phishing_detected_at}초")
    else:
        print("\n✅ 보이스피싱 의심 없음")

    return lines, predictions, phishing_detected_at

# -------------------------------
# 7. 시각화
# -------------------------------
def plot_predictions(lines, predictions, phishing_detected_at=None):
    time_intervals = [i * 10 for i in range(len(lines))]

    plt.figure(figsize=(10, 5))
    plt.plot(time_intervals, predictions, marker='o', linestyle='-', color='red', label='예측 확률')
    plt.title("피싱 통화 확률 시각화")
    plt.xlabel("시간 (초)")
    plt.ylabel("피싱 통화 확률")
    plt.ylim(0, 1)
    plt.grid(True)

    if phishing_detected_at is not None:
        plt.axvline(x=phishing_detected_at, color='blue', linestyle='--', label='탐지 시점')
        plt.text(phishing_detected_at + 1, 0.85, '⚠ 탐지됨', color='blue')

    plt.legend()
    plt.tight_layout()
    plt.show()

# -------------------------------
# 8. 전체 파이프라인 실행 함수
# -------------------------------
def process_mp3(mp3_file_path):
    # 1. mp3 → wav
    wav_file_path = mp3_to_wav(mp3_file_path, 'temp_audio.wav')

    # 2. whisper로 음성 인식
    transcribed_text = transcribe_audio_with_whisper(wav_file_path)

    # 3. 모델 및 토크나이저 로딩
    model, tokenizer = load_model_and_tokenizer()

    # 4. 예측 및 누적 위험도 분석
    lines, predictions, phishing_detected_at = run_prediction_with_risk(transcribed_text, model, tokenizer)

    # 5. 시각화
    plot_predictions(lines, predictions, phishing_detected_at)

    return transcribed_text, predictions

# -------------------------------
# 9. 실행
# -------------------------------
mp3_file_path = '/content/drive/MyDrive/voicephishing1/example.mp3'
transcribed_text, predictions = process_mp3(mp3_file_path)


In [ ]:
import os
from pydub import AudioSegment
import whisper
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import string

# -------------------------------
# 1. mp3 → wav 변환
# -------------------------------
def mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")
    print(f"✅ MP3 → WAV 변환 완료: {wav_file_path}")
    return wav_file_path

# -------------------------------
# 2. Whisper 음성 인식
# -------------------------------
def transcribe_audio_with_whisper(wav_file_path):
    model = whisper.load_model("large")
    result = model.transcribe(wav_file_path)
    print("\n📝 음성 인식 결과:")
    print(result['text'])
    return result['text']

# -------------------------------
# 3. 전처리 함수
# -------------------------------
def preprocess_text(text, tokenizer, max_length=30):
    text = text.translate(str.maketrans('', '', string.punctuation))  # 문장부호 제거
    sequences = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    return padded

# -------------------------------
# 4. 예측 함수
# -------------------------------
def predict_with_lstm(model, text, tokenizer):
    padded_text = preprocess_text(text, tokenizer)
    prediction = model.predict(padded_text, verbose=0)
    return prediction[0][0]

# -------------------------------
# 5. 모델 & 토크나이저 로딩
# -------------------------------
def load_model_and_tokenizer():
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer



# -------------------------------
# 6. 누적 평균 기반 위험도 분석
# -------------------------------
def run_prediction_with_cumulative_average(text, model, tokenizer, threshold=0.7):
    print("\n📊 누적 평균 위험도 분석 시작")
    lines = [line.strip() for line in text.split('.') if line.strip()]  # 문장 분할
    predictions = []
    cumulative_sum = 0.0
    phishing_detected_at = None

    for i, line in enumerate(lines):
        prob = predict_with_lstm(model, line, tokenizer)
        predictions.append(prob)
        cumulative_sum += prob
        cumulative_avg = cumulative_sum / (i + 1)

        print(f"{i+1:02d}. \"{line}\" → 확률: {prob:.4f}, 누적 평균: {cumulative_avg:.4f}")

        if phishing_detected_at is None and cumulative_avg > threshold:
            phishing_detected_at = i * 10  # 시간 단위 가정

    if phishing_detected_at is not None:
        print(f"\n⚠️ 보이스피싱 의심 탐지 시점: {phishing_detected_at}초 (누적 평균 기반)")
    else:
        print("\n✅ 보이스피싱 의심 없음")

    return lines, predictions, phishing_detected_at

# -------------------------------
# 7. 시각화 (변경 없음)
# -------------------------------
def plot_predictions(lines, predictions, phishing_detected_at=None):
    time_intervals = [i * 10 for i in range(len(lines))]

    plt.figure(figsize=(10, 5))
    plt.plot(time_intervals, predictions, marker='o', linestyle='-', color='red', label='예측 확률')
    plt.title("피싱 통화 확률 시각화")
    plt.xlabel("시간 (초)")
    plt.ylabel("피싱 통화 확률")
    plt.ylim(0, 1)
    plt.grid(True)

    if phishing_detected_at is not None:
        plt.axvline(x=phishing_detected_at, color='blue', linestyle='--', label='탐지 시점')
        plt.text(phishing_detected_at + 1, 0.85, '⚠ 탐지됨', color='blue')

    plt.legend()
    plt.tight_layout()
    plt.show()

# -------------------------------
# 8. 전체 파이프라인 실행 함수
# -------------------------------
def process_mp3(mp3_file_path):
    # 1. mp3 → wav
    wav_file_path = mp3_to_wav(mp3_file_path, 'temp_audio.wav')

    # 2. whisper로 음성 인식
    transcribed_text = transcribe_audio_with_whisper(wav_file_path)

    # 3. 모델 및 토크나이저 로딩
    model, tokenizer = load_model_and_tokenizer()

    # 4. 예측 및 누적 평균 기반 분석
    lines, predictions, phishing_detected_at = run_prediction_with_cumulative_average(
        transcribed_text, model, tokenizer
    )

    # 5. 시각화
    plot_predictions(lines, predictions, phishing_detected_at)

    return transcribed_text, predictions


# -------------------------------
# 9. 실행
# -------------------------------
mp3_file_path = '/content/drive/MyDrive/voicephishing1/example.mp3'
transcribed_text, predictions = process_mp3(mp3_file_path)



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import string

# -------------------------------
# 1. 텍스트 전처리
# -------------------------------
def preprocess_text(text, tokenizer, max_length=30):
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequences, maxlen=max_length, truncating='post', padding='post')
    return padded_sequence

# -------------------------------
# 2. LSTM 예측
# -------------------------------
def predict_with_lstm(model, text, tokenizer):
    text = text.translate(str.maketrans('', '', string.punctuation))  # 문장부호 제거
    processed_text = preprocess_text(text, tokenizer)
    prediction = model.predict(processed_text)
    return prediction

# -------------------------------
# 3. 모델 및 Tokenizer 로딩
# -------------------------------
def load_model_and_tokenizer():
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

# -------------------------------
# 4. 대화 스크립트 기반 처리
# -------------------------------
def process_script(script_lines):
    model, tokenizer = load_model_and_tokenizer()
    time_intervals = []
    predictions = []

    for i, line in enumerate(script_lines):
        prediction = predict_with_lstm(model, line, tokenizer)
        time_intervals.append(i * 10)  # 10초 간격 기준
        predictions.append(prediction[0][0])
        print(f"{i+1:02d}. \"{line}\" → 예측 확률: {prediction[0][0]:.4f}")

    return time_intervals, predictions

# -------------------------------
# 5. 테스트할 보이스피싱인 척하는 대화 스크립트
# -------------------------------
script_lines = [
    "여보세요",  # 피해자
    "고객님 맞으시죠",  # 피싱범
    "네, 누구세요?",  # 피해자
    "지금 잠깐 통화 가능하신가요",  # 피싱범
    "네, 말씀하세요",  # 피해자
    "저는 금융보호센터 상담팀에 있는 김수현이라고 합니다",  # 피싱범
    "금융보호센터요?",  # 피해자
    "다름이 아니라 고객님 명의로 개설된 계좌에서 최근 이상 거래가 감지돼서요",  # 피싱범
    "이상 거래요?",  # 피해자
    "혹시 오늘 오전에 휴대폰으로 이체하신 적 있으신가요",  # 피싱범
    "아뇨 그런 적 없어요",  # 피해자
    "확인해보니까 고객님 계좌에서 부산에서 접속한 기록이 떠서요",  # 피싱범
    "저 부산에 없는데요",  # 피해자
    "그럼 본인 확인만 간단하게 도와드릴게요",  # 피싱범
    "어떻게 해요?",  # 피해자
    "은행은 국민은행 사용하고 계시죠",  # 피싱범
    "네",  # 피해자
    "다른 건 필요 없고요",  # 피싱범
    "이름이랑 계좌번호 앞 네자리만 말씀해주시면 됩니다",  # 피싱범
    "홍길동, 1234요",  # 피해자 (예시)
    "네 확인되었습니다",  # 피싱범
    "지금 바로 보호조치 들어가고 있으니까요",  # 피싱범
    "잠시 후에 인증번호가 문자로 갈 텐데",  # 피싱범
    "받으시는 대로 그 번호만 다시 말씀해주세요",  # 피싱범
    "아 네 알겠습니다",  # 피해자
    "절차 끝나면 자동으로 보호조치 완료됩니다",  # 피싱범
    "감사합니다",  # 피해자
    "불편드려 죄송합니다",  # 피싱범
]

# -------------------------------
# 6. 처리 및 시각화
# -------------------------------
time_intervals, predictions = process_script(script_lines)

# [수정 1] 0초에서 0을 추가하여 그래프가 0,0에서 시작하도록 만듦
time_intervals = [0] + [t + 10 for t in time_intervals]
predictions = [0] + predictions

# [수정 2] x축 눈금을 10초 단위로 모두 표시
plt.figure(figsize=(10, 6))
plt.plot(time_intervals, predictions, marker='o', color='r', linestyle='-', label='피싱 통화 확률')
plt.title("스크립트에 따른 피싱 통화 확률")
plt.xlabel("시간 (초)")
plt.ylabel("피싱 통화 확률")
plt.grid(True)
plt.legend()

plt.xticks(np.arange(0, max(time_intervals) + 10, 10))

plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import string

# -------------------------------
# 1. 텍스트 전처리
# -------------------------------
def preprocess_text(text, tokenizer, max_length=30):
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequences, maxlen=max_length, truncating='post', padding='post')
    return padded_sequence

# -------------------------------
# 2. LSTM 예측
# -------------------------------
def predict_with_lstm(model, text, tokenizer):
    text = text.translate(str.maketrans('', '', string.punctuation))  # 문장부호 제거
    processed_text = preprocess_text(text, tokenizer)
    prediction = model.predict(processed_text, verbose=0)
    return prediction

# -------------------------------
# 3. 모델 및 Tokenizer 로딩
# -------------------------------
def load_model_and_tokenizer():
    model = tf.keras.models.load_model('/content/drive/MyDrive/voicephishing1/lstm_model0514.keras')
    with open('/content/drive/MyDrive/voicephishing1/tokenizer0514.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

# -------------------------------
# 4. 대화 스크립트 기반 처리 (누적 위험도 포함)
# -------------------------------
def process_script(script_lines, base_threshold=0.5, detection_threshold=3.0):
    model, tokenizer = load_model_and_tokenizer()
    time_intervals = []
    predictions = []
    cumulative_risk = 0.0
    phishing_detected_at = None

    print("\n[예측 결과 및 누적 위험도]\n")
    for i, line in enumerate(script_lines):
        prediction = predict_with_lstm(model, line, tokenizer)
        prob = prediction[0][0]
        predictions.append(prob)
        time_intervals.append(i * 10)

        # 누적 위험도 계산
        if prob > base_threshold:
            cumulative_risk += (prob - base_threshold)
        else:
            cumulative_risk = max(0, cumulative_risk - (base_threshold - prob))

        print(f"{i+1:02d}. \"{line}\" → 확률: {prob:.4f}, 누적 위험도: {cumulative_risk:.4f}")

        if phishing_detected_at is None and cumulative_risk >= detection_threshold:
            phishing_detected_at = i * 10

    if phishing_detected_at is not None:
        print(f"\n⚠️ 보이스피싱 의심 탐지 시점: {phishing_detected_at}초")
    else:
        print("\n✅ 보이스피싱 의심되지 않음")

    return time_intervals, predictions, phishing_detected_at

# -------------------------------
# 5. 테스트할 대화 스크립트
# -------------------------------
script_lines = [
    "여보세요",  # 피해자
    "고객님 맞으시죠",  # 피싱범
    "네, 누구세요?",  # 피해자
    "지금 잠깐 통화 가능하신가요",  # 피싱범
    "네, 말씀하세요",  # 피해자
    "저는 금융보호센터 상담팀에 있는 김수현이라고 합니다",  # 피싱범
    "금융보호센터요?",  # 피해자
    "다름이 아니라 고객님 명의로 개설된 계좌에서 최근 이상 거래가 감지돼서요",  # 피싱범
    "이상 거래요?",  # 피해자
    "혹시 오늘 오전에 휴대폰으로 이체하신 적 있으신가요",  # 피싱범
    "아뇨 그런 적 없어요",  # 피해자
    "확인해보니까 고객님 계좌에서 부산에서 접속한 기록이 떠서요",  # 피싱범
    "저 부산에 없는데요",  # 피해자
    "그럼 본인 확인만 간단하게 도와드릴게요",  # 피싱범
    "어떻게 해요?",  # 피해자
    "은행은 국민은행 사용하고 계시죠",  # 피싱범
    "네",  # 피해자
    "다른 건 필요 없고요",  # 피싱범
    "이름이랑 계좌번호 앞 네자리만 말씀해주시면 됩니다",  # 피싱범
    "홍길동, 1234요",  # 피해자 (예시)
    "네 확인되었습니다",  # 피싱범
    "지금 바로 보호조치 들어가고 있으니까요",  # 피싱범
    "잠시 후에 인증번호가 문자로 갈 텐데",  # 피싱범
    "받으시는 대로 그 번호만 다시 말씀해주세요",  # 피싱범
    "아 네 알겠습니다",  # 피해자
    "절차 끝나면 자동으로 보호조치 완료됩니다",  # 피싱범
    "감사합니다",  # 피해자
    "불편드려 죄송합니다",  # 피싱범
]

# -------------------------------
# 6. 처리 및 시각화
# -------------------------------
time_intervals, predictions, phishing_detected_at = process_script(script_lines)

plt.figure(figsize=(10, 6))
plt.plot(time_intervals, predictions, marker='o', color='r', linestyle='-', label='피싱 통화 확률')
plt.title("스크립트에 따른 피싱 통화 확률")
plt.xlabel("시간 (초)")
plt.ylabel("피싱 통화 확률")
plt.grid(True)
plt.ylim(0, 1)

# 탐지 시점 시각화
if phishing_detected_at is not None:
    plt.axvline(x=phishing_detected_at, color='blue', linestyle='--', label='탐지 시점')
    plt.text(phishing_detected_at + 1, 0.9, '⚠ 탐지됨', color='blue')

plt.legend()
plt.show()